# Disk Artifacts in Memory Part 2
The next source of disk artifacts we will expore are related to the artifacts in memory related to the file system New Technology File Systems or NTFS. NTFS is the standard file system for Windows NT and later and is a significant improvement over File Application Table or FAT file system. The improvements over FAT include the following:

* More metadata can be stored
* Performance with advanced data structures
* Security controls
* Storage capacity
* Journaling

A key artifact in NTFS file systems is the Master File Talbe which is refered to as the `$MFT` which is the resident file on disk, below is the `$MFT` file from a disk viewed in EnCase.  

<img src="images/MFT.png">

The `$MFT` is a NTFS metadata file containing an entry for every file and directory. Important information contained in the `$MFT` for incident response analysis are the following:

* Name
* Inode Number (File identifier)
* Standard Information Attributes (SIA)
* Filename Information Attributes (FIA)
* Data Attributes

The `$MFT` is similar to a database and has record entires for every file and folder. A view of one of these records is below:

<img src="images/MFTRecordEntry.png">

The `$MFT` record entry consists of a Header which is `FILE0` and each entry is a Length if 1024 bytes. The `$MFT` record entry can contain the contents of the whole file if it is small enough and will not reside on disk and just in the `$MFT`. The `$MFT` has five data points of interest to incident response and forensic analysts. These attributes are shown in the figure below:

<img src="images/MFTRecordDataPoints.png">

The first attribute is the header followed by the Standard Information Attribute (SIA). The SIA contains the date information seen in the Windows Explorer interface and are refered to as MAC times or Modified, Accessed, Written and Created dates. These dates are shown in the figure below form a `$MFT` record entry:

<img src="images/MFTDates.png">

The identifier for the SIA is `10 00 00 00`. The exact offset for these data points are below:

|Byte Range|Description|Essential|
|----------|-----------|---------|
|0-7|File Creation|No|
|8-15|File Last Written|No|
|16-23|File Last Modified|No|
|24-31|File Last Accessed|No|


There are other dates stored within the `$MFT` record which are not visable from Windows Explorer or editable from Windows API's. These dates are refered to as File Name Attributes or FNA.  The FNA stores the files name and parent directory information and stored the folowoing dates related to the file:

* Filename Creation
* Filename Modified
* Filename Last Written
* Filename Last Accessed

These dates are highlighted in a `$MFT` record entry in the figure below:

<img src="images/MFTFNDATES.png">

The offsets to parse these dates and their function are in the table below:

|Byte Range|Description|Essential|
|----------|-----------|---------|
|0-7|File Reference of parent directory|No|
|8-15|File Creation|No|
|16-23|File Last Written|No|
|24-31|File Last Modified|No|
|32-39|File Last Accessed|No|
|66+|Name|Yes/No|

The identifier for the FNA is `30 00 00 00`. Now that you understand the basics of `$MFT` related data we can start to identify areas of interest for analysis. The first place to start when reviewing is where the Standard Information Attribute dates do not match the Filename Information Attributes. This can produce a lot of false positives don’t be surprised, and focus on your time frame of initional compromise. You can also timeline on the FNA data points. Comparing these two data points can help to defeat time stomping, since the FNA cannot be modifed through Windows API's. You can also review MFT Slack which can contain file data from previous resident files and also in memore review the whole resident file. The `$MFT` can also hold previous INDX Buffer data that can help with identifing exfiltrated data.

## MFTPARSER
To view `$MFT` record entries in memory execute the volatility plugin `mftparser` as shown in the code block below:

In [1]:
python ../volatility/vol.py -f ../APT.img --profile=WinXPSP2x86 mftparser

Volatility Foundation Volatility Framework 2.6
Scanning for MFT entries and building directory, this can take a while
***************************************************************************
MFT entry found at offset 0x35000
Attribute: In Use & File
Record Number: 12724
Link count: 1


$STANDARD_INFORMATION
Creation                       Modified                       MFT Altered                    Access Date                    Type
------------------------------ ------------------------------ ------------------------------ ------------------------------ ----
2008-04-14 12:00:00 UTC+0000 2007-08-13 22:32:16 UTC+0000   2008-10-29 16:38:28 UTC+0000   2008-08-11 21:43:13 UTC+0000   Compressed & Archive

$FILE_NAME
Creation                       Modified                       MFT Altered                    Access Date                    Name/Path
------------------------------ ------------------------------ ------------------------------ ------------------------------ ---------
2007-08